In [1]:
import tensorflow as tf
import pandas as pd


In [2]:
df = pd.read_csv('Bias_correction_ucl.csv')
df.head()

,station,Date,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,...,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation,Next_Tmax,Next_Tmin
0,1.0,2013-06-30,28.7,21.4,58.255688,91.116364,28.074101,23.006936,6.818887,69.451805,...,0.0,0.0,0.0,37.6046,126.991,212.3350,2.7850,5992.895996,29.1,21.2
1,2.0,2013-06-30,31.9,21.6,52.263397,90.604721,29.850689,24.035009,5.691890,51.937448,...,0.0,0.0,0.0,37.6046,127.032,44.7624,0.5141,5869.312500,30.5,22.5
2,3.0,2013-06-30,31.6,23.3,48.690479,83.973587,30.091292,24.565633,6.138224,20.573050,...,0.0,0.0,0.0,37.5776,127.058,33.3068,0.2661,5863.555664,31.1,23.9
3,4.0,2013-06-30,32.0,23.4,58.239788,96.483688,29.704629,23.326177,5.650050,65.727144,...,0.0,0.0,0.0,37.6450,127.022,45.7160,2.5348,5856.964844,31.7,24.3
4,5.0,2013-06-30,31.4,21.9,56.174095,90.155128,29.113934,23.486480,5.735004,107.965535,...,0.0,0.0,0.0,37.5507,127.135,35.0380,0.5055,5859.552246,31.2,22.5


In [3]:
df.drop('Date', inplace=True, axis=1)
df.dropna(inplace=True)
df.head()

,station,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,LDAPS_CC1,...,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation,Next_Tmax,Next_Tmin
0,1.0,28.7,21.4,58.255688,91.116364,28.074101,23.006936,6.818887,69.451805,0.233947,...,0.0,0.0,0.0,37.6046,126.991,212.3350,2.7850,5992.895996,29.1,21.2
1,2.0,31.9,21.6,52.263397,90.604721,29.850689,24.035009,5.691890,51.937448,0.225508,...,0.0,0.0,0.0,37.6046,127.032,44.7624,0.5141,5869.312500,30.5,22.5
2,3.0,31.6,23.3,48.690479,83.973587,30.091292,24.565633,6.138224,20.573050,0.209344,...,0.0,0.0,0.0,37.5776,127.058,33.3068,0.2661,5863.555664,31.1,23.9
3,4.0,32.0,23.4,58.239788,96.483688,29.704629,23.326177,5.650050,65.727144,0.216372,...,0.0,0.0,0.0,37.6450,127.022,45.7160,2.5348,5856.964844,31.7,24.3
4,5.0,31.4,21.9,56.174095,90.155128,29.113934,23.486480,5.735004,107.965535,0.151407,...,0.0,0.0,0.0,37.5507,127.135,35.0380,0.5055,5859.552246,31.2,22.5


In [4]:
target = df[['Next_Tmax', 'Next_Tmin']]
features = df.drop(['Next_Tmax', 'Next_Tmin'], axis=1)



In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
feature_array = scaler.fit_transform(features)
features = pd.DataFrame(feature_array, columns=features.columns)


In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(features.shape[1],), name='Input_layer'),
    tf.keras.layers.Dense(16, name="Dense1"), # activation='relu'),
    tf.keras.layers.Dense(8, name="Dense2"), # activation='relu'),
    tf.keras.layers.Dense(4, name="Dense3"), # activation='relu'),
    tf.keras.layers.Dense(2, name="OutputLayer")
])
model.compile(tf.optimizers.RMSprop(0.001), loss='mse', metrics=['accuracy'])
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Dense1 (Dense)              (None, 16)                368       
                                                                 
 Dense2 (Dense)              (None, 8)                 136       
                                                                 
 Dense3 (Dense)              (None, 4)                 36        
                                                                 
 OutputLayer (Dense)         (None, 2)                 10        
                                                                 
Total params: 550
Trainable params: 550
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(x=features.to_numpy(), y=target.to_numpy(), epochs=50, callbacks=[tensorboard_callback], validation_split=0.2)

Epoch 1/50
190/190 [==============================] - 1s 2ms/step - loss: 293.9465 - accuracy: 0.9483 - val_loss: 16.2295 - val_accuracy: 1.0000
Epoch 2/50
190/190 [==============================] - 0s 1ms/step - loss: 8.9116 - accuracy: 1.0000 - val_loss: 5.1322 - val_accuracy: 1.0000
Epoch 3/50
190/190 [==============================] - 0s 1ms/step - loss: 4.2210 - accuracy: 1.0000 - val_loss: 3.8049 - val_accuracy: 1.0000
Epoch 4/50
190/190 [==============================] - 0s 1ms/step - loss: 3.1070 - accuracy: 1.0000 - val_loss: 3.3336 - val_accuracy: 1.0000
Epoch 5/50
190/190 [==============================] - 0s 1ms/step - loss: 2.6702 - accuracy: 1.0000 - val_loss: 3.1490 - val_accuracy: 1.0000
Epoch 6/50
190/190 [==============================] - 0s 1ms/step - loss: 2.4160 - accuracy: 1.0000 - val_loss: 3.1262 - val_accuracy: 1.0000
Epoch 7/50
190/190 [==============================] - 0s 1ms/step - loss: 2.2525 - accuracy: 1.0000 - val_loss: 3.0029 - val_accuracy: 1.0000
Epo

In [9]:

loss = model.evaluate(features.to_numpy(), target.to_numpy())
print('loss: ', loss)

238/238 [==============================] - 0s 670us/step - loss: 1.6333 - accuracy: 1.0000
loss:  [1.6333050727844238, 1.0]
